In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm

In [2]:
df=pd.read_csv('../Travel_Dataset/Travel details dataset.csv')
df.head()

,Trip ID,Destination,Start date,End date,Duration (days),Traveler name,Traveler age,Traveler gender,Traveler nationality,Accommodation type,Accommodation cost,Transportation type,Transportation cost
0,1,"London, UK",5/1/2023,5/8/2023,7.0,John Smith,35.0,Male,American,Hotel,1200,Flight,600
1,2,"Phuket, Thailand",6/15/2023,6/20/2023,5.0,Jane Doe,28.0,Female,Canadian,Resort,800,Flight,500
2,3,"Bali, Indonesia",7/1/2023,7/8/2023,7.0,David Lee,45.0,Male,Korean,Villa,1000,Flight,700
3,4,"New York, USA",8/15/2023,8/29/2023,14.0,Sarah Johnson,29.0,Female,British,Hotel,2000,Flight,1000
4,5,"Tokyo, Japan",9/10/2023,9/17/2023,7.0,Kim Nguyen,26.0,Female,Vietnamese,Airbnb,700,Train,200


In [3]:
# there are three rows we don't consider
    # 2 of which have no information expect for a trip id (72, 128)
    # 1 of which does not have a transportation type and transportation cost (83) 
num_rows = df.shape[0]
print(f"orginal number of rows: {num_rows}")

# get rid of rows with missing data
df = df.dropna()

# unique_Trip_ID = df['Trip ID'].unique()
# print(unique_Trip_ID)

num_rows = df.shape[0]
print(f"new number of rows: {num_rows}")

orginal number of rows: 139
new number of rows: 136


# data that needs to be cleaned
* Destination
* Accommodation cost
* Transportation cost

In [4]:
# Get unique destination country names after cleaning
unique_Destination = df['Destination'].unique()
print(unique_Destination)

# Extracting unique country names from 'Destination' column
unique_destinations = df['Destination'].apply(lambda x: x.split(', ')[-1]).unique()

# Function to clean and standardize destination country names
def clean_destination(unique_destinations):
    country_map = {
        'Thai': 'Thailand',
        'AUS': 'Australia',
        'Aus': 'Australia',
        'SA': 'South Africa',
        'UK': 'United Kingdom',
        'USA': 'United States',
        'UAE': 'United Arab Emirates',
        'NZ': 'New Zealand',
        'Hawaii': 'United States'  # Assuming Hawaii is in the United States
        # Add more mappings as needed
    }

    # Replace country abbreviations or alternate representations with full country names
    return country_map.get(unique_destinations, unique_destinations)

# Apply cleaning function to the 'Destination' column
df['Destination'] = df['Destination'].apply(lambda x: clean_destination(x.split(', ')[-1]))

# Get unique country names from the cleaned 'Destination' column
unique_countries = df['Destination'].unique()

# Print the cleaned list of unique countries
print(f"\nCleaned unique countries:\n{unique_countries}")



['London, UK' 'Phuket, Thailand' 'Bali, Indonesia' 'New York, USA'
 'Tokyo, Japan' 'Paris, France' 'Sydney, Australia'
 'Rio de Janeiro, Brazil' 'Amsterdam, Netherlands'
 'Dubai, United Arab Emirates' 'Cancun, Mexico' 'Barcelona, Spain'
 'Honolulu, Hawaii' 'Berlin, Germany' 'Marrakech, Morocco'
 'Edinburgh, Scotland' 'Paris' 'Bali' 'London' 'Tokyo' 'New York' 'Sydney'
 'Rome' 'Bangkok' 'Hawaii' 'Barcelona' 'Japan' 'Thailand' 'France'
 'Australia' 'Brazil' 'Greece' 'Egypt' 'Mexico' 'Italy' 'Spain' 'Canada'
 'New York City, USA' 'Bangkok, Thailand' 'Vancouver, Canada'
 'Sydney, AUS' 'Seoul, South Korea' 'Los Angeles, USA' 'Rome, Italy'
 'Cape Town' 'Cape Town, SA' 'Sydney, Aus' 'Bangkok, Thai' 'Phuket, Thai'
 'Dubai' 'Seoul' 'Rio de Janeiro' 'Amsterdam' 'Phuket' 'Santorini'
 'Phnom Penh' 'Athens, Greece' 'Cape Town, South Africa'
 'Auckland, New Zealand']

Cleaned unique countries:
['United Kingdom' 'Thailand' 'Indonesia' 'United States' 'Japan' 'France'
 'Australia' 'Brazil' 'Netherland

In [5]:
unique_Accommodation_Cost = df['Accommodation cost'].unique()
print(unique_Accommodation_Cost)

df['Accommodation cost'] = df['Accommodation cost'].str.replace(r'\$|USD', '', regex=True)

# Get unique values after removing the '$' sign
unique_Accommodation_Cost = df['Accommodation cost'].unique()
print(f"\nCleaned Accommodation Cost: \n{unique_Accommodation_Cost}")

['1200' '800' '1000' '2000' '700' '1500' '500' '900' '2500' '3000' '1400'
 '600' '$900 ' '$1,500 ' '$1,200 ' '$600 ' '$1,000 ' '$700 ' '$400 '
 '$1,400 ' '$2,000 ' '$1,100 ' '$800 ' '$200 ' '$150 ' '$500 ' '$180 '
 '$350 ' '2200' '400' '1100' '300' '1300' '1800' '500 USD' '1000 USD'
 '800 USD' '1200 USD' '400 USD' '600 USD' '900 USD' '1500 USD' '700 USD'
 '300 USD' '200 USD' '$300 ' '200' '5000' '7000' '6000' '4000' '8000']

Cleaned Accommodation Cost: 
['1200' '800' '1000' '2000' '700' '1500' '500' '900' '2500' '3000' '1400'
 '600' '900 ' '1,500 ' '1,200 ' '600 ' '1,000 ' '700 ' '400 ' '1,400 '
 '2,000 ' '1,100 ' '800 ' '200 ' '150 ' '500 ' '180 ' '350 ' '2200' '400'
 '1100' '300' '1300' '1800' '1000 ' '1200 ' '1500 ' '300 ' '200' '5000'
 '7000' '6000' '4000' '8000']


In [6]:
unique_Transportation_Cost = df['Transportation cost'].unique()
print(unique_Transportation_Cost)


df['Transportation cost'] = df['Transportation cost'].str.replace(r'\$|USD', '', regex=True)

unique_Transportation_Cost = df['Transportation cost'].unique()
print(f"\nCleaned Transportation Cost: \n{unique_Transportation_Cost}")

['600' '500' '700' '1000' '200' '800' '1200' '100' '400' '150' '$400 '
 '$700 ' '$150 ' '$800 ' '$100 ' '$600 ' '$80 ' '$500 ' '$300 ' '$50 '
 '$120 ' '$75 ' '900' '50' '$200 ' '$250 ' '$20 ' '300' '800 USD'
 '200 USD' '500 USD' '700 USD' '300 USD' '600 USD' '400 USD' '1000 USD'
 '100 USD' '350 USD' '150 USD' '$1,200 ' '$900 ' '$1,500 ' '$1,000 ' '250'
 '2500' '1500' '2000' '3000']

Cleaned Transportation Cost: 
['600' '500' '700' '1000' '200' '800' '1200' '100' '400' '150' '400 '
 '700 ' '150 ' '800 ' '100 ' '600 ' '80 ' '500 ' '300 ' '50 ' '120 ' '75 '
 '900' '50' '200 ' '250 ' '20 ' '300' '1000 ' '350 ' '1,200 ' '900 '
 '1,500 ' '1,000 ' '250' '2500' '1500' '2000' '3000']
